<H2 style="background-color: #444444">Things to install</H2>
sudo apt install x11-xserver-utils libdbus-1-3 libxkbcommon-x11-0 libxcb-icccm4 libxcb-image0 libxcb-keysyms1 libxcb-randr0 libxcb-render-util0 

sudo apt install libxcb-xinerama0 libxcb-xinput0 libxcb-xfixes0

mamba install napari[all] matplotlib pyqt pyyaml bioformats_jar

pip install aicsimageio[all]

pip install ome-zarr

pip install gunpowder

pip install readlif>=0.6.4

mamba update tifffile


In [1]:
import os
import shutil
import numpy as np
import zarr
import yaml

from aicsimageio import AICSImage, writers
from ome_zarr.io import parse_url, ZarrLocation
from ome_zarr.writer import write_image

from matplotlib.pyplot import imshow

In [2]:
# utility functions
# define file path
def dataPath(
    dirPath: str = "/",
    fileName: str = None,
    fileID: int = 0,
    outputMode: int = 0  # output options: 0: file path, 1:splitted path, 2: file list
):

    fileList = os.listdir(dirPath)
    if fileName is None:
        fileName = fileList[fileID]
    fileName, filExt = os.path.splitext(fileName)
    filePath = os.path.join(dirPath, fileName + filExt)

    assert outputMode in [0, 1, 2], "outputMode must be 0 or 1"
    match outputMode:
        case 0:  # return file path
            return filePath
        case 1:  # return file path and file extension
            return dirPath, fileName, filExt
        case 2:  # return names in a file directory
            return fileList
        case _:
            raise ValueError(
                "outputMode must be 0-2 (0: file path, 1:splitted path, 2: file list")


# load data into an AICSImage object
def dataLoader(filepath: str = None):
    assert filepath != None, "filepath is None"
    assert os.path.exists(filepath), "file does not exist"
    img = AICSImage(filepath)
    return img


# save file into a zarr dircetory
def newZarr(
    pathZarr: str = "/",
    img: AICSImage = None,
    remove: bool = True
):

    assert img != None, "img is None"
    if os.path.exists(pathZarr):
        if remove:
            shutil.rmtree(pathZarr)
        else:
            raise OSError("file exists")
    os.mkdir(pathZarr)
    store = parse_url(pathZarr, mode="w").store
    image = img.data
    root = zarr.group(store=store)
    axes = img.dims.order.lower()
    imgShape = img.shape
    write_image(
        image=image,
        group=root,
        axes=axes,
        storage_options=dict(chunks=(1, imgShape[3], imgShape[4]))
    )


# convert all files in a directory to zarr
def zarrConverter(
    path: str = "/",
    remove: bool = True,
    dataTypes: str = ".tif"
):

    assert os.path.exists(path), "file does not exist"

    fileList = dataPath(path, outputMode=2)
    assert len(fileList) > 0, "file directory is empty"

    filePaths = []
    for file in fileList:
        if file.endswith(dataTypes):
            filePath = dataPath(path, file)
            dirPath, fileName, _ = dataPath(path, file, outputMode=1)
            zarrPath = os.path.join(dirPath, (fileName + '.zarr'))
            filePaths.append(zarrPath)

            img = dataLoader(filePath)
            newZarr(
                pathZarr=zarrPath,
                img=img,
                remove=remove
            )
    assert len(filePaths) > 0, "no files are converted"
    return filePaths

In [ ]:
# example run convert all tif files under data directory
zarrConverter(
    path="/mnt/c/data/",
    remove=True,
    dataTypes=".tif"
)

In [ ]:
# process the first image in the directory
DIR = "/mnt/c/data/spheres/images/"
filePath = dataPath(DIR, fileID=0)
newZarr(
    pathZarr=filePath,
    img=AICSImage(filePath),
    remove=True
)

In [ ]:
# process the first image in the directory
DIR = "/mnt/c/data/spheres/objects/"
NAME = "0001_sphere.tiff"
filePath = dataPath(DIR, NAME)
newZarr(
    pathZarr=filePath,
    img=AICSImage(filePath),
    remove=True
)